https://github.com/rajpurkar # 데이터
    
https://medium.com/@eyfydsyd97/bert-for-question-answer-fine-tuning%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-by-pytorch-fbe15fdef330

In [1]:
import torch
from transformers import BertTokenizer

In [7]:
tokenizer= BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

# Use SquAD 1.1 data
#input은 [CLS]+[context]+[SEP]+[question]+[SEP]+[PAD]

def text_processing(data, Max_len = 192):
    input_ids = []
    token_type_ids = []
    attn_mask_ids = []
    label_start = []
    label_end = []
    
    for i in range(len(data)): 
        label_st = 0
        label_ed = 0 
        context = data['context'][i]
        que = data['question'][i]
        ans = data['answer'][i]
        sent1 = tokenizer.encode(context, add_special_tokens=False, max_length=Max_len)

    if len(sent1) < Max_len:
        sent2 = tokenizer.encode(que, add_special_tokens=False)
        sent3 = tokenizer.encode(ans, add_special_tokens=False)

        len_ans = len(sent3)
        for j in range(len(sent1)):
            if sent1[j: j + len_ans] == sent3:
                label_st = j + 1
                label_ed = j + len_ans

        length = len(sent1) + len(sent2) + 3
        ids = [101] + sent1+ [102] + sent2 + [102] + [0]*(Max_len - length)
        ids = ids[:Max_len]
        type_ids = [0] * (len(sent1) + 2) + [1]*(Max_len - len(sent1) - 2) 
        type_ids = type_ids[:Max_len]
        attn_ids = [1]*length + [0]*(Max_len - length)
        attn_ids = attn_ids[:Max_len]

        input_ids.append(torch.tensor(ids, dtype=torch.long))
        token_type_ids.append(torch.tensor(type_ids, dtype=torch.long))
        attn_mask_ids.append(torch.tensor(attn_ids, dtype=torch.long))
        label_start.append(torch.tensor(label_st, dtype=torch.long))
        label_end.append(torch.tensor(label_ed, dtype=torch.long))
  
    return input_ids, token_type_ids, attn_mask_ids, label_start, label_end

In [ ]:
class BERT_QA_dataset(Dataset):
    def __init__(self, input_ids, token_type_ids, attn_mask_ids, label_start, label_end):
        self.input = input_ids
        self.token_type = token_type_ids
        self.attn_mask = attn_mask_ids
        self.label_start = label_start
        self.label_end = label_end
    def __len__(self):
        return len(self.input)
    
    def __getitem(self, idx):
        x = self.input[idx]
        y = self.token_type[idx]
        z = self.attn_mask[idx]
        label_s = self.label_start[idx].view(-1)
        label_e = self.label_end[idx].view(-1)
        
        return x, y, z, label_s, label_e
    
train_dataset = BERT_QA_dataset(input_ids, token_type_ids, attn_mask_idx, label_start, label_end)
valid_dataset = BERT_QA_dataset(input_ids_v, token_type_ids_v, attn_mask_ids_v, label_start_v, label_end_v)

train_dataloader = DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=Batch_size, shuffle=True)

In [10]:
bert = BertModel.from_pretrained('bert-base-uncased')

class Bert_QA(nn.Module):
    def __init__(self):
        super().__init()
        self.bert = bert
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(778, 2)
        
    def froward(self, input_ids, token_type_ids, attn_mask_idx):
        output, _ = self.bert(input_ids, attention_mask = attn_mask_ids, token_type_ids=token_type_ids)
        x = self.dropout(output)
        logits = self.linear(x)
        start_logits, end_logits = logits.split(1, dim=-1)
        
        return start_logits, end_logits

NameError: name 'BertModel' is not defined

In [ ]:
def loss_func(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [ ]:
def train_one_epoch(data_loader, model, optimizer, device, loss_fn):  
    model.train()
    tk0 = tqdm(data_loader, total=len(data_loader))
    total_loss = 0.0
    
    for bi, d in enumerate(tk0):
        input_ids, token_type_ids, attn_mask_ids, label_start, label_end = d
        input_ids = input_ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
        label_start = label_start.to(device, dtype=torch.long)
        label_end = label_end.to(device, dtype=torch.long)

        model.zero_grad()
        start_logits, end_logits = model(input_ids, token_type_ids, attn_mask_ids)
        loss = loss_fn(start_logits, end_logits, label_start,  label_end)
        total_loss += loss.item()
        if bi % 100 ==0:
            print(f"loss:{loss}")
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(data_loader) 
    print("Average training loss: {0:.2f}".format(avg_train_loss))  
    
def eval_one_epoch(data_loader, model,  device, loss_fn):

    model.eval()
    tk0 = tqdm(data_loader, total=len(data_loader))
    fin_targets_st = []
    fin_outputs_st = []
    fin_targets_ed = []
    fin_outputs_ed = []
    
    with torch.no_grad():

    for bi, d in enumerate(tk0):
        input_ids, token_type_ids, attn_mask_ids, label_start, label_end = d
        input_ids = input_ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
        label_start = label_start.to(device, dtype=torch.long)
        label_end = label_end.to(device, dtype=torch.long)

        logits_start, logits_end = model(input_ids, token_type_ids, attn_mask_ids)
         loss = loss_fn(logits_start, logits_end, label_start,  label_end)

        logits_start = logits_start.detach().cpu().numpy()
        label_start = label_start.detach().cpu().numpy()
        logits_end = logits_end.detach().cpu().numpy()
        label_end = label_end.detach().cpu().numpy()
        pred_start = np.argmax(logits_start, axis=1).flatten()
        pred_end = np.argmax(logits_end, axis=1).flatten()

        fin_targets_st.extend(label_start.tolist())
        fin_outputs_st.extend(pred_start.tolist()) 
        fin_targets_ed.extend(label_end.tolist())
        fin_outputs_ed.extend(pred_end.tolist())  
        
    return fin_outputs_st, fin_targets_st, fin_outputs_ed, fin_targets_ed


def fit(train_dataloader, valid_dataloader, EPOCHS=3):
    bert_QA=Bert_QA() #model
    bert_QA=bert_QA.to(device)
    loss_fn = loss_func #loss
    optimizer = torch.optim.AdamW(bert_QA.parameters(),lr=lr) #optimizer

    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, bert_QA, optimizer, device, loss_fn)
        print('EVAL')
        outputs_st, targets_st, outputs_ed, targets_ed = eval_one_epoch(valid_dataloader, bert_QA,  device, loss_fn)
        targets_st, targets_ed = np.array(targets_st), np.array(targets_ed)
        auc_st = accuracy_score(targets_st, outputs_st)
        auc_ed = accuracy_score(targets_ed, outputs_ed)
        auc = (auc_st + auc_ed)/2
        
        print(f"auc_st;{auc_st}") 
        print(f"auc_ed;{auc_ed}") 
        print(f"auc;{auc}")  
    
fit(train_dataloader, valid_dataloader)    